In [31]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [32]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,11.20,74,96,9.60,SH,1690964456
1,1,west island,-12.1568,96.8225,26.99,65,40,8.23,CC,1690964457
2,2,olinda,-8.0089,-34.8553,25.85,78,40,3.09,BR,1690964458
3,3,haiku-pauwela,20.9219,-156.3051,24.47,84,20,6.69,US,1690964458
4,4,baracoa,20.3467,-74.4958,26.84,89,95,1.53,CU,1690964459


In [4]:
# Configure the map plot
map_plot = city_data_df.hvplot.points(x='Lng', y='Lat', size='Humidity', geo=True, tiles='OSM',
                                      hover_cols=['Humidity'], frame_height=400)

# Display the map
map_plot

: 

: 

In [34]:

# Narrow down cities that fit criteria and drop any results with null values
max_temp_c_criteria = 27  
min_temp_c_criteria = 21  
wind_speed_criteria = 4.5  
cloudiness_criteria = 0  
# Drop any rows with null values
filtered_cities_df = city_data_df[
    ((city_data_df['Max Temp'] < max_temp_c_criteria) |
    (city_data_df['Max Temp'] > min_temp_c_criteria)) &
    (city_data_df['Wind Speed'] < wind_speed_criteria) &
    (city_data_df['Cloudiness'] == cloudiness_criteria)
]

# Display sample data
filtered_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,tromso,69.6496,18.9570,18.01,55,0,2.57,NO,1690964459
7,7,jamestown,42.0970,-79.2353,12.80,96,0,0.45,US,1690964461
18,18,longyearbyen,78.2186,15.6401,9.91,76,0,2.06,SJ,1690964468
20,20,zuera,41.8677,-0.7898,26.95,42,0,0.73,ES,1690964469
25,25,machado,-21.6747,-45.9197,12.12,78,0,0.32,BR,1690964473
...,...,...,...,...,...,...,...,...,...,...
551,551,muncar,-8.4333,114.3333,27.64,68,0,3.05,ID,1690964831
552,552,villa regina,-39.1000,-67.0667,10.55,22,0,2.18,AR,1690964832
556,556,lethbridge,49.7000,-112.8186,18.69,58,0,1.03,CA,1690964697
566,566,piui,-20.4653,-45.9581,13.58,68,0,0.39,BR,1690964432


In [37]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
5,tromso,NO,69.6496,18.9570,55,
7,jamestown,US,42.0970,-79.2353,96,
18,longyearbyen,SJ,78.2186,15.6401,76,
20,zuera,ES,41.8677,-0.7898,42,
25,machado,BR,-21.6747,-45.9197,78,
...,...,...,...,...,...,...
551,muncar,ID,-8.4333,114.3333,68,
552,villa regina,AR,-39.1000,-67.0667,22,
556,lethbridge,CA,49.7000,-112.8186,58,
566,piui,BR,-20.4653,-45.9581,68,


In [36]:
# Set parameters to search for a hotel
radius = 10000 
params = {
    "type": "hotel", 
    "limit": 1,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = {"type": "circle", "radius": radius, "lat": lat, "lon": lng}
    params["bias"] = {"lat": lat, "lon": lng}
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address =  requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tromso - nearest hotel: No hotel found
jamestown - nearest hotel: No hotel found
longyearbyen - nearest hotel: No hotel found
zuera - nearest hotel: No hotel found
machado - nearest hotel: No hotel found
griffith - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
hamilton - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
bredasdorp - nearest hotel: No hotel found
hermanus - nearest hotel: No hotel found
harrisburg - nearest hotel: No hotel found
kodiak - nearest hotel: No hotel found
butte - nearest hotel: No hotel found
fort bragg - nearest hotel: No hotel found
las brenas - nearest hotel: No hotel found
port elizabeth - nearest hotel: No hotel found
sumbawa besar - nearest hotel: No hotel found
utiel - nearest hotel: No hotel found
kruisfontein - nearest hotel: No hotel found
broken hill - nearest hotel: No hotel found
east london - nearest hotel: No hotel found
rio grande - nearest hotel: No hotel found
illizi - nearest 

,City,Country,Lat,Lng,Humidity,Hotel Name
5,tromso,NO,69.6496,18.9570,55,No hotel found
7,jamestown,US,42.0970,-79.2353,96,No hotel found
18,longyearbyen,SJ,78.2186,15.6401,76,No hotel found
20,zuera,ES,41.8677,-0.7898,42,No hotel found
25,machado,BR,-21.6747,-45.9197,78,No hotel found
...,...,...,...,...,...,...
551,muncar,ID,-8.4333,114.3333,68,No hotel found
552,villa regina,AR,-39.1000,-67.0667,22,No hotel found
556,lethbridge,CA,49.7000,-112.8186,58,No hotel found
566,piui,BR,-20.4653,-45.9581,68,No hotel found


In [38]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(x='Lng', y='Lat', c='Humidity', size='Humidity', colormap='viridis',
                                  hover_cols=['City', 'Country', 'Hotel Name'], alpha=0.7,
                                  title='Hotel Locations and Humidity',
                                  xlabel='Longitude', ylabel='Latitude')


# Display the map
map_plot

:Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)